In [1]:
import json
import datetime
import time
import requests
import nbimporter
import random
import os
import psutil
import psycopg2
# from multiprocessing import Process, Pool
import Insert_py
import Get_py
import Tor
import Proxy_Gatherer as pg
import Init_database as indb
import email_alert as reminder
import warnings
warnings.filterwarnings("ignore")

Importing Jupyter notebook from Insert_py.ipynb
Importing Jupyter notebook from Get_py.ipynb
Importing Jupyter notebook from Tor.ipynb
Importing Jupyter notebook from Proxy_Gatherer.ipynb
Importing Jupyter notebook from Init_database.ipynb
Importing Jupyter notebook from email_alert.ipynb


In [2]:
#--------INITIALIZING CONNECTION TO DATABASE--------#
db_name="db_teld"
user_name="postgres"
password = "ecalscraping"
host="127.0.0.1"
port="5432"
conn = indb.init_db(db_name, user_name, password, host, port)

In [3]:
#------- Gathering proxies for visiting teld.cn -------##
def proxy():
    hdr = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36',
           'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8'}
    gatherproxy = pg.ProxyGatherer(hdr)
    proxies_xici = gatherproxy.getproxies_xici()
    proxies_elite = gatherproxy.getproxies_elite()
#     proxies_usproxy = gatherproxy.getproxies_usproxy()
    # t = time.time()
#     proxiesXici_pool = gatherproxy.proxies_cleanup(proxies_xici) # Could also specify timeout limit you prefer, 
                                                                    #format--(connect time, read time)
#     proxiesElite_pool = gatherproxy.proxies_cleanup(proxies_elite)
    proxies = proxies_xici+proxies_elite
#     return proxies
    return proxies

In [4]:
def write_error(error_log, pillcode, error_msg):
    """
        Input:
            error_log - file, write error messages into designated file
    """
    error_log.write(pillcode)
    error_log.write(str(error_msg))
    error_log.write('\n')

In [5]:
def header_gen():
    import numpy as np
    app_version = np.random.randint(1,5)
    os_version = np.random.randint(0,3)+round(np.random.randint(0,3)*0.1,1)
    user_ag = "Teld/3.{a}.0 (iPhone; iOS 10.{o}; Scale/2.00)".format(a=app_version,o=os_version)
    device = "app_version=3.{a}.0&os_version=10.{o}&client=ios&device_name=iPhone 6s (A1633/A1688/A1691/A1700)&device_id=38CD294C-CC36-4998-9074-C7F798EE6087&city_code=4403&city_name=%E6%B7%B1%E5%9C%B3&lat=0.000000&lng=0.000000&network=wifi&location_city_name=".format(a=app_version,o=os_version)
    hdr = {
        'accept': "*/*",
        'user-agent': user_ag,
        'cookie': "TELDAppID=",
        'content-length': "111",
        'connection': "keep-alive",
        'accept-encoding': "gzip, deflate",
        'device': device,
        'host': "basesg.teld.cn",
        'content-type': "application/x-www-form-urlencoded",
        'cache-control': "no-cache"
    }
    return hdr

In [6]:
def ChargeHist_header_gen():
    import numpy as np
    app_version = np.random.randint(1,5)
    os_version = np.random.randint(0,3)+round(np.random.randint(0,3)*0.1,1)
    user_ag = "Teld/3.{a}.0 (iPhone; iOS 10.{o}; Scale/2.00)".format(a=app_version,o=os_version)
    device = "app_version=3.{a}.0&os_version=10.{o}&client=ios&device_name=iPhone 6s (A1633/A1688/A1691/A1700)&device_id=38CD294C-CC36-4998-9074-C7F798EE6087&city_code=4403&city_name=%E6%B7%B1%E5%9C%B3&lat=0.000000&lng=0.000000&network=wifi&location_city_name=".format(a=app_version,o=os_version)
    hdr = {
        'accept': "*/*",
        'user-agent': user_ag,
        'cookie': "TELDAppID=",
        'content-length': "250",
        'connection': "keep-alive",
        'accept-encoding': "gzip, deflate",
        'device': device,
        'host': "basesg.teld.cn",
        'content-type': "application/x-www-form-urlencoded",
        'cache-control': "no-cache"
    }
    return hdr
ChargeHist_hdr = ChargeHist_header_gen()

In [7]:
def user_act_AIclean(user_act):
    pileName = str(user_act['PileName'].encode('utf-8'))
    if user_act['userNickName'] is not None:
        userNickName = str(user_act['userNickName'].encode('utf-8'))
    else:
        userNickName = None
    if user_act['carTypeName'] is not None:
        carTypeName = str(user_act['carTypeName'].encode('utf-8'))
    else:

Python 2

    File
    Edit
    View
    Insert
    Cell
    Kernel
    Widgets
    Help

import json

import datetime

import time

import requests

import nbimporter

import random

import os

import psutil

import psycopg2

# from multiprocessing import Process, Pool

import Insert_py

import Get_py

import Tor

import Proxy_Gatherer as pg

import Init_database as indb

import email_alert as reminder

import warnings

warnings.filterwarnings("ignore")

Importing Jupyter notebook from Insert_py.ipynb
Importing Jupyter notebook from Get_py.ipynb
Importing Jupyter notebook from Tor.ipynb
Importing Jupyter notebook from Proxy_Gatherer.ipynb
Importing Jupyter notebook from Init_database.ipynb
Importing Jupyter notebook from email_alert.ipynb

#--------INITIALIZING CONNECTION TO DATABASE--------#

db_name="db_teld"

user_name="postgres"

password = "ecalscraping"

host="127.0.0.1"

port="5432"

conn = indb.init_db(db_name, user_name, password, host, port)

#------- Gathering proxies for visiting teld.cn -------##

def proxy():

    hdr = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36',

           'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8'}

    gatherproxy = pg.ProxyGatherer(hdr)

    proxies_xici = gatherproxy.getproxies_xici()

    proxies_elite = gatherproxy.getproxies_elite()

#     proxies_usproxy = gatherproxy.getproxies_usproxy()

    # t = time.time()

#     proxiesXici_pool = gatherproxy.proxies_cleanup(proxies_xici) # Could also specify timeout limit you prefer, 

                                                                    #format--(connect time, read time)

#     proxiesElite_pool = gatherproxy.proxies_cleanup(proxies_elite)

    proxies = proxies_xici+proxies_elite

#     return proxies

    return proxies

def write_error(error_log, pillcode, error_msg):

    """

        Input:

            error_log - file, write error messages into designated file

    """

    error_log.write(pillcode)

    error_log.write(str(error_msg))

    error_log.write('\n')

def header_gen():

    import numpy as np

    app_version = np.random.randint(1,5)

    os_version = np.random.randint(0,3)+round(np.random.randint(0,3)*0.1,1)

    user_ag = "Teld/3.{a}.0 (iPhone; iOS 10.{o}; Scale/2.00)".format(a=app_version,o=os_version)

    device = "app_version=3.{a}.0&os_version=10.{o}&client=ios&device_name=iPhone 6s (A1633/A1688/A1691/A1700)&device_id=38CD294C-CC36-4998-9074-C7F798EE6087&city_code=4403&city_name=%E6%B7%B1%E5%9C%B3&lat=0.000000&lng=0.000000&network=wifi&location_city_name=".format(a=app_version,o=os_version)

    hdr = {

        'accept': "*/*",

        'user-agent': user_ag,

        'cookie': "TELDAppID=",

        'content-length': "111",

        'connection': "keep-alive",

        'accept-encoding': "gzip, deflate",

        'device': device,

        'host': "basesg.teld.cn",

        'content-type': "application/x-www-form-urlencoded",

        'cache-control': "no-cache"

    }

    return hdr

def ChargeHist_header_gen():

    import numpy as np

    app_version = np.random.randint(1,5)

    os_version = np.random.randint(0,3)+round(np.random.randint(0,3)*0.1,1)

    user_ag = "Teld/3.{a}.0 (iPhone; iOS 10.{o}; Scale/2.00)".format(a=app_version,o=os_version)

    device = "app_version=3.{a}.0&os_version=10.{o}&client=ios&device_name=iPhone 6s (A1633/A1688/A1691/A1700)&device_id=38CD294C-CC36-4998-9074-C7F798EE6087&city_code=4403&city_name=%E6%B7%B1%E5%9C%B3&lat=0.000000&lng=0.000000&network=wifi&location_city_name=".format(a=app_version,o=os_version)

    hdr = {

        'accept': "*/*",

        'user-agent': user_ag,

        'cookie': "TELDAppID=",

        'content-length': "250",

        'connection': "keep-alive",

        'accept-encoding': "gzip, deflate",

        'device': device,

        'host': "basesg.teld.cn",

        'content-type': "application/x-www-form-urlencoded",

        'cache-control': "no-cache"

    }

    return hdr

ChargeHist_hdr = ChargeHist_header_gen()

def user_act_AIclean(user_act):

    pileName = str(user_act['PileName'].encode('utf-8'))

    if user_act['userNickName'] is not None:

        userNickName = str(user_act['userNickName'].encode('utf-8'))

    else:

        userNickName = None

    if user_act['carTypeName'] is not None:

        carTypeName = str(user_act['carTypeName'].encode('utf-8'))

    else:

        carTypeName = None

    startTime = user_act['chargeTime']

    #AI detect startTime

    if len(startTime) < 6:

    #if in the expression of hours:

        if hour_uni in startTime:

            idx_hour = len(startTime)-3

            hours = int(startTime[:idx_hour])

            startTime = startTime = datetime.datetime.now() - datetime.timedelta(hours=hours)

#                     startTime = '{:%Y-%m-%d %H:%M:%S}'.format(startTime)+"-07"

            startTime = '{:%Y-%m-%d %H:%M}'.format(startTime)+"-07"

    #if in the expression of minutes:

        if minute_uni in startTime:

            idx_min = len(startTime)-3

            minutes = int(startTime[:idx_min])

            startTime = startTime = datetime.datetime.now() - datetime.timedelta(minutes=minutes)

#                     startTime = '{:%Y-%m-%d %H:%M:%S}'.format(startTime)+"-07"

            startTime = '{:%Y-%m-%d %H:%M}'.format(startTime)+"-07"

    elif len(startTime) >= 6:

        startTime = '2017-' + startTime + '-07'   

    return startTime, pileName, carTypeName, userNickName

# ##------- Every once a while runs it update station and pile fixed info -------##

# #--------------------------------------------#

# #------- Gather all stations  -------# ****NEED TO BE FIXED 09.17.2017

# station_info = Get_py.get_allStations(hdr)

# print(len(station_info))

# for station in station_info:

#     stationId = str(station['id'])  

# #     print(stationId)

#     #------- Gather individual stations details -------#

#     station_detailedInfo = Get_py.get_oneStation_APP(hdr, stationId)

#     conn = Insert_py.insert_singStation_details(conn, station_detailedInfo)

#     #------- Gather individual pile code -------#

#     piles_info = Get_py.get_piles(stationId, proxies_pool, hdr)

#     conn = Insert_py.insert_pilecode(conn, stationId, piles_info)

# #------- Gather inidividual pile details -------#

# #------- Request stationid which has user activities from db -------#

try:

    cur = conn.cursor()

    cur.execute("select distinct stationid from tbl_app_useractivity;")

    stationWithAct_list = cur.fetchall()

    cur.close()

except psycopg2.Error as e:

    print(e)

print(len(stationWithAct_list))

# #------- Request pilecode from db for states -------#

staid_list = []

for staid in stationWithAct_list:

    staid_list.append(staid[0])

try:

    cur = conn.cursor()

    sta_list = ['2b3a35d4-cb94-4b8f-a47a-bbb054754066', '1d02640a-7f14-4fc3-baaf-60754a38fff7',

                'ac6cb946-5fff-491b-859e-55cc5f701657', 'dd4a3ddc-e1b0-4e2e-b7ca-faef627f4e66',

                'b134a159-16bd-4f5d-af1b-cd42599a3bab', '33a3c031-0a3a-417d-ba1b-53549bb08dc2', 

                'e4758edb-1183-4c51-887b-a52ca7abef3d', '13bdab71-110b-4448-a3c4-5c3113183546']

    #1. beijing hangkong jingmi jixie yanjiusuo chongdianzhan --'2b3a35d4-cb94-4b8f-a47a-bbb054754066',

    #2. beijing xijiao bingguan chongdianzhan -- '1d02640a-7f14-4fc3-baaf-60754a38fff7

    #3. beijing yangguanglicheng jiudian chongdianzhan --'ac6cb946-5fff-491b-859e-55cc5f701657',

    #4. beijing rongtao gongsi chongdianzhan -- '32d317a9-3d0c-4458-8357-89a2b3b1cf94'

    #5. beijing huairou qu yanshui lu -- 'dd4a3ddc-e1b0-4e2e-b7ca-faef627f4e66'

    

    #6. shenzhen hengjiang dasha chongdianzhan -- 'b134a159-16bd-4f5d-af1b-cd42599a3bab'

    #7. shenzhen yinxing gongyeyuan chongdianzhan -- '33a3c031-0a3a-417d-ba1b-53549bb08dc2'

    #8. shenzhe wanke jiulongshan chongdianzhan -- 'e4758edb-1183-4c51-887b-a52ca7abef3d'

    #9. shenzhen jingyuan dasha chongdianzhan -- '13bdab71-110b-4448-a3c4-5c3113183546'

    stmt = "select pilecode from tbl_APP_pilecode where staid in %s"

    cur.execute(stmt, (tuple(sta_list),))

    

#     stmt = "select pilecode from tbl_APP_pilecode where staid = 'dd4a3ddc-e1b0-4e2e-b7ca-faef627f4e66'"

#     cur.execute(stmt)

#     cur.execute("select pilecode from tbl_APP_pilecode")

    pillcode_list = cur.fetchall()

    cur.close()

except psycopg2.Error as e:

    print(e)

print(len(pillcode_list))

​

# # ------- Requests stationID from db for user charging activities -------#

try:

    cur = conn.cursor()

    cur.execute("select stationid from tbl_app_eachstations;")

    station_list = cur.fetchall()

    cur.close()

except psycopg2.Error as e:

    print(e)

print(len(station_list))

​

# reportTime = datetime.datetime.now() + datetime.timedelta(minutes=15)

# i = 0

# for pile in pilecode_list:

#     #If encounter error, try one more request with a different proxy

#     try:

#         pile_detailedInfo, proxies_pool = Get_py.get_onePile_APP(F, hdr, pile[0], proxies_pool, time_out=1000)

#     except ValueError as e:

#         try:

#             pile_detailedInfo, proxies_pool = Get_py.get_onePile_APP(F, hdr, pile[0], proxies_pool, time_out=1000)

#         except ValueError as e:

#             pile_detailedInfo = None

#         except (requests.exceptions.ChunkedEncodingError) as e:

#             pile_detailedInfo = None

#     except (requests.exceptions.ChunkedEncodingError) as e:

#         try:

#             pile_detailedInfo, proxies_pool = Get_py.get_onePile_APP(F, hdr, pile[0], proxies_pool, time_out=1000)

#         except ValueError as e:

#             pile_detailedInfo = None

#         except (requests.exceptions.ChunkedEncodingError) as e:

#             pile_detailedInfo = None

#     if pile_detailedInfo is not None:

#         conn = Insert_py.insert_singPile_details(conn, pile_detailedInfo)

#     else:

#         print("Running None Func.")

#         conn = Insert_py.insert_singPile_none(conn, pile_detailedInfo, pile[0])

#     i += 1

#     if datetime.datetime.now() >= reportTime:

#         reportTime = datetime.datetime.now() + datetime.timedelta(minutes=15) 

#         print("Successfully run {} piles".format(i))

# #211 Running None Func.

# #71585 piles.

# #------- Request Shenzhen charging pilecode from db for states -------#

try:

    cur = conn.cursor()

#     stmt1 = "select stationid from tbl_app_eachstations where tbl_app_eachstations.location && ST_MakeEnvelope(115.37916, 39.47148, 117.15623, 40.81726, 4326);"    

    stmt1 = "select stationid from tbl_app_eachstations where tbl_app_eachstations.location && ST_MakeEnvelope(113.85955, 22.50563, 114.17335, 22.69960, 4326) intersect select distinct stationid from tbl_app_useractivity;"

    cur.execute(stmt1)

    sta_list = cur.fetchall()

    sta_list_clean = []

    for sta in sta_list:

        sta_list_clean.append(sta[0])

    stmt2 = "select pilecode from tbl_APP_pilecode where staid in %s"

    cur.execute(stmt2, (tuple(sta_list_clean),))

    shenzhen_pillcode_list = cur.fetchall()

    cur.close()

except psycopg2.Error as e:

    print(e)

print(len(shenzhen_pillcode_list))

4859
86
8866
926

# # #------- Update user activity -------#

# i = 0

# fail = 0

# hour_uni = u'\u5c0f\u65f6\u524d'

# minute_uni = u'\u5206\u949f\u524d'

# # proxies = proxy()

# # proxy_idx = random.randint(0,len(proxies)-1) 

# # os.environ["https_proxy"]=proxies[proxy_idx]['https']

# # os.environ["http_proxy"]=proxies[proxy_idx]['http']

# activity_dict = {}

# for idx, stationID in enumerate(station_list):

#     stationID = stationID[0]

# #     if len(proxies) == 1: #proxy pool dried out

# #         proxies = proxy()

    

# #     proxy_idx = random.randint(0,len(proxies)-1)

# #     os.environ["https_proxy"]=proxies[proxy_idx]['https']

# #     os.environ["http_proxy"]=proxies[proxy_idx]['http']

    

#     try_num = 2

# #     while try_num:

# #         if try_num < 2:

# #             print("Retry")

# #         try:

# #             chargeList_detailedInfo = Get_py.get_chargelist_APP(stationID,ChargeHist_hdr)

# #         except:

# #             print("Changing proxy")

# #             proxies.pop(proxy_idx)

# #             if len(proxies) == 1:

# #                 proxies = proxy()

# #                 proxy_idx = random.randint(0,len(proxies)-1)

# #             else:

# #                 proxy_idx = random.randint(0,len(proxies)-1)

# #             os.environ["https_proxy"]=proxies[proxy_idx]['https']

# #             os.environ["http_proxy"]=proxies[proxy_idx]['http']

# #         try_num -= 1

# #         if not try_num:

# #             chargeList_detailedInfo=[]

# #             fail += 1

# #             print('fail +', fail, ' index', idx)

                

#     try:

#         chargeList_detailedInfo = Get_py.get_chargelist_APP(stationID,ChargeHist_hdr)

#     except:

#         try:

# #             proxies.pop(proxy_idx)

# #             proxy_idx = random.randint(0,len(proxies)-1)

# #             os.environ["https_proxy"]=proxies[proxy_idx]['https']

# #             os.environ["http_proxy"]=proxies[proxy_idx]['http']

#             chargeList_detailedInfo = Get_py.get_chargelist_APP(stationID,ChargeHist_hdr)

#         except:

# #             proxies.pop(proxy_idx)

#             chargeList_detailedInfo=[]

#             fail += 1

#             print('fail +', fail, ' index', idx)

#             pass

#     if len(chargeList_detailedInfo) == 0:

#         i += 1

#     else:

#         activity_dict['{}'.format(stationID)] = chargeList_detailedInfo

# print(str(i) + " stations do not have activity history.")

# print(str(fail) + " connections failed attempt.")

# print(datetime.datetime.now())

# fromaddr = "296257740.jz@gmail.com"

# psw = "15710725968crzzq"

# toaddr = "hustlejanton@berkeley.edu"

# subject="Courtesy Email"

# body="Program terminated by time - 16046 user activity"

# reminder.email_courtesy(fromaddr, toaddr, psw, subject, body)

# print("Email sent successfully.")

# for stationID in activity_dict.keys():

#     chargeList_detailedInfo = activity_dict[stationID]

#     for user_act in chargeList_detailedInfo:

#         startTime, pileName, carTypeName, userNickName = user_act_AIclean(user_act)

#         conn = Insert_py.insert_userActivity_info(conn, startTime, pileName, stationID, carTypeName, userNickName)

def pillState_scrap(pillcode): 

    """

        Input:

            pillCode - string, each charging pile has its own pile code

        Output:

            pill_detailedInfo - dictionary of pill state details

    """

    hdr = header_gen()

    time_out = 20

    F = None

    proxies_pool = None

    #If encounter error, try one more request with a different proxy

    try:

        pill_detailedInfo, proxies_pool = Get_py.get_onePill_APP(hdr, pillcode, F, proxies_pool, time_out=time_out)

#     except ValueError as e:

#         if F is not None:

#             write_error(F, pillcode, e)

#         try:

#             pill_detailedInfo, proxies_pool = Get_py.get_onePill_APP(hdr, pillcode, F, proxies_pool, time_out=time_out)

#         except ValueError as e:

#             if F is not None:

#                 write_error(F, pillcode, e)

#             pill_detailedInfo = None

#         except (requests.exceptions.ChunkedEncodingError) as e:

#             if F is not None:

#                 write_error(F, pillcode, e)

# #             else:

#             pill_detailedInfo = None

#     except (requests.exceptions.ChunkedEncodingError) as e:

#         if F is not None:

#             write_error(F, pillcode, e)

#         try:

#             pill_detailedInfo, proxies_pool = Get_py.get_onePill_APP(hdr, pillcode, F, proxies_pool, time_out=time_out)

#         except ValueError as e:

#             if F is not None:

#                 write_error(F, pillcode, e)

#             pill_detailedInfo = None

#         except (requests.exceptions.ChunkedEncodingError) as e:

#             if F is not None:

#                 write_error(F, pillcode, e)

# #         else:

#             pill_detailedInfo = None

# #     return pill_detailedInfo, proxies_pool

    except:

        pill_detailedInfo = None

    return pill_detailedInfo

def insert_state(outputs):

    for pill_detailedInfo in outputs:

        if pill_detailedInfo is not None:

            db_name="db_teld"

            user_name="postgres"

            password = "ecalscraping"

            host="127.0.0.1"

            port="5432"

            conn = indb.init_db(db_name, user_name, password, host, port)

            pillCode = str(pill_detailedInfo['pillCode'])

            stateCode = str(pill_detailedInfo['stateCode'])

            stateCodeV3 = str(pill_detailedInfo['stateCodeV3'])

            if pill_detailedInfo['stateName'] is not None:

                stateName = str(pill_detailedInfo['stateName'].encode('utf-8'))

            else:

                stateName = ''

            if pill_detailedInfo['stateNameV3'] is not None:

                stateNameV3 = str(pill_detailedInfo['stateNameV3'].encode('utf-8'))

            else:

                stateNameV3 = ''

            conn = Insert_py.insert_state_info(conn, pillCode, stateCode, stateName, stateCodeV3, stateNameV3)

            conn.close()

## ----- Test for single looping of multiprocessing post_request and insert -----##

​

# pid = os.getpid()

# py = psutil.Process(pid)

# memoryUse = py.memory_info()[0]/2.**20 #in MB

# print('Before runnning script, memory use:', memoryUse, 'MB')

# # loop_num = 1

# # max_proc =10000

# pool = Pool(200)

# t = time.time()

# if __name__ == '__main__':

#     results = [pool.apply_async(pillState_scrap, (pillcode[0],)) for pillcode in pillcode_list]

#     print(len(results))

#     outputs = [p.get() for p in results]

# print("success, time costs is " + str(time.time()-t)) 

​

# pid = os.getpid()

# py = psutil.Process(pid)

# memoryUse = py.memory_info()[0]/2.**20 #in MB

# print('After runnning script,memory use:', memoryUse, 'MB')

for pillcode in pillcode_list[70:]:

    print(pillcode[0])

4403060016101
4403060072102
4403060072110
4403060072112
4403060072104
4403060072106
4403060072107
4403060072113
4403060072111
4403060072103
4403060072108
4403060072109
4403060072101
4403060072115
4403060072105
4403060072114

# os.environ["https_proxy"]="https://165.84.167.54:8080"

# os.environ["http_proxy"]="http://165.84.167.54:8080"

# pool = Pool(processes=2)

# startTime = datetime.datetime.now()

endTime = datetime.datetime.now() + datetime.timedelta(hours=168)

# proxyUpdateTime = datetime.datetime.now() + datetime.timedelta(hours=3)

# tmpproxy = {'http': 'http://101.78.219.178:8080', 'https': 'https://101.78.219.178:8080'}

# os.environ["https_proxy"]=tmpproxy['https']

# os.environ["http_proxy"]=tmpproxy['http']

# proxies = proxy()

# proxy_idx = random.randint(0,len(proxies)-1) 

# os.environ["https_proxy"]=proxies[proxy_idx]['https']

# os.environ["http_proxy"]=proxies[proxy_idx]['http']

​

fromaddr = "296257740.jz@gmail.com"

psw = "15710725968crzzq"

toaddr = "hustlejanton@berkeley.edu"

subject="Courtesy Email"

while True:

    if datetime.datetime.now() >= endTime:

        body="Program terminated by time - 16043"

        reminder.email_courtesy(fromaddr, toaddr, psw, subject, body)

        break

#     Tor.renewTor()

#     Tor.connectTor()

#     Tor.showmyIp()

    print("waking up!")

#     if datetime.datetime.now() >= proxyUpdateTime: #hit update time

#         proxies = proxy()

#         proxyUpdateTime = datetime.datetime.now() + datetime.timedelta(hours=3)

    if len(proxies) <= 1: #proxy pool dried out before next update time

        proxies = proxy()

        proxy_idx = random.randint(0,len(proxies)-1) 

        os.environ["https_proxy"]=proxies[proxy_idx]['https']

        os.environ["http_proxy"]=proxies[proxy_idx]['http']

    t = time.time()

    outputs=[]

#     results = [pool.apply_async(pillState_scrap, (pillcode[0],)) for pillcode in pillcode_list]

#     outputs = [p.get() for p in results]

    missing_data = 0

    for pillcode in pillcode_list:

#         Tor.connectTor()

        try_num = 2

        while try_num:

            if try_num < 2:

                print("Retry")

            pill_detailedInfo = pillState_scrap(pillcode[0])

            if pill_detailedInfo is not None:

                outputs.append(pill_detailedInfo)

                break

            if pill_detailedInfo is None:

                print("Changing proxy")

                proxies.pop(proxy_idx)

                if len(proxies) == 1:

                    proxies = proxy()

                    proxy_idx = random.randint(0,len(proxies)-1)

                else:

                    proxy_idx = random.randint(0,len(proxies)-1)

                os.environ["https_proxy"]=proxies[proxy_idx]['https']

                os.environ["http_proxy"]=proxies[proxy_idx]['http']

            try_num -= 1

            if not try_num:

                missing_data += 1

    if endTime - datetime.datetime.now() <= datetime.timedelta(hours=1):

        body="Program less than one hour left."

        reminder.email_courtesy(fromaddr, toaddr, psw, subject, body)

    print("Takes "+str(time.time()-t)+" seconds")

    print("Missing {} of pills data".format(missing_data))

    insert_state(outputs)

    print("sleeping...")

    time.sleep(random.randint(600, 900))

waking up!
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbaeb250>, 'Connection to 92.42.107.227 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb7e650>, 'Connection to 92.42.107.228 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dba13090>, 'Connection to 171.116.229.98 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 504 Gateway Time-out',)))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Too many open connections',)))
failed
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbafbb10>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Takes 350.39707613 seconds
Missing 7 of pills data
sleeping...
waking up!
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 400 Bad Request',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dba23490>, 'Connection to 122.183.139.104 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dba23090>, 'Connection to 41.76.150.154 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 501 Not Implemented',)))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db62b850>, 'Connection to 61.6.120.52 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
Changing proxy
Takes 399.778263092 seconds
Missing 6 of pills data
sleeping...
waking up!
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db377810>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db377c10>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db5a55d0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db5a58d0>, 'Connection to 122.183.139.98 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db5a5ed0>, 'Connection to 143.202.210.217 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db5a5f10>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db5a56d0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dba23450>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db5a5a50>, 'Connection to 113.93.224.54 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry

Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 504 Gateway Time-out',)))
failed
Changing proxy
Retry
Takes 570.345552206 seconds
Missing 9 of pills data
sleeping...
waking up!
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db850450>, 'Connection to 186.223.93.248 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3ef5d0>, 'Connection to 78.111.92.59 timed out. (connect timeout=20)'))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4022d0>, 'Connection to 137.74.254.242 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db402f50>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db402310>, 'Connection to 177.47.213.44 timed out. (connect timeout=20)'))
failed
Changing proxy
Takes 518.749086857 seconds
Missing 5 of pills data
sleeping...
waking up!
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 403 Forbidden',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3f8910>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Retry
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Retry
Takes 437.544389009 seconds
Missing 6 of pills data
sleeping...
waking up!
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db5a53d0>, 'Connection to 197.248.223.230 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Service Unavailable',)))
failed
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3fd0d0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db6daa90>, 'Connection to 122.183.139.104 timed out. (connect timeout=20)'))
failed
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
Changing proxy
Takes 525.196864128 seconds
Missing 7 of pills data
sleeping...
waking up!
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb7e650>, 'Connection to 14.153.52.232 timed out. (connect timeout=20)'))
failed
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db377910>, 'Connection to 60.168.75.216 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry

('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db410890>, 'Connection to 60.214.155.243 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db410890>, 'Connection to 113.88.13.161 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db410d10>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db410a50>, 'Connection to 115.46.99.97 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db410c50>, 'Connection to 174.34.177.250 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dba23710>, 'Connection to 185.108.215.247 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dba23510>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
Changing proxy
Takes 931.090389967 seconds
Missing 13 of pills data
sleeping...
waking up!
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db37a150>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db45add0>, 'Connection to 182.253.34.232 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3f4cd0>, 'Connection to 1.196.160.149 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3ef3d0>, 'Connection to 125.65.177.52 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Takes 443.984788895 seconds
Missing 5 of pills data
sleeping...
waking up!
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Too many open connections',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3fd990>, 'Connection to 60.214.155.243 timed out. (connect timeout=20)'))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db5a5f10>, 'Connection to 115.46.89.49 timed out. (connect timeout=20)'))
failed
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3fd2d0>, 'Connection to 94.253.79.35 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbfec890>, 'Connection to 176.119.74.107 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbfeca90>, 'Connection to 14.221.164.213 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dba23350>, 'Connection to 14.221.164.213 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry

Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4102d0>, 'Connection to 183.33.192.21 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'SSL23_GET_SERVER_HELLO', 'unknown protocol')],)",),))
failed
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3ef5d0>, 'Connection to 117.80.28.219 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3efc50>, 'Connection to 148.251.225.239 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3ef290>, 'Connection to 113.77.240.214 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3ef610>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db402b10>, 'Connection to 122.183.139.98 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db402810>, 'Connection to 123.138.89.133 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db402910>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db402250>, 'Connection to 1.196.160.149 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db45a910>, 'Connection to 115.46.89.49 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Takes 789.075012922 seconds
Missing 12 of pills data
sleeping...
waking up!
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9a56d0>, 'Connection to 60.168.75.216 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db6da210>, 'Connection to 91.225.228.156 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db6dabd0>, 'Connection to 36.77.173.24 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db6dae10>, 'Connection to 149.202.0.60 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db6da0d0>, 'Connection to 182.253.177.80 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db6da150>, 'Connection to 62.16.21.221 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db6da290>, 'Connection to 113.88.13.161 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db6daad0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db6b1310>, 'Connection to 14.153.52.232 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db410710>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3774d0>, 'Connection to 80.51.81.222 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Takes 493.430454016 seconds
Missing 7 of pills data

sleeping...
waking up!
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db6dab10>, 'Connection to 35.229.53.125 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db6dab90>, 'Connection to 181.129.16.170 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db6daa90>, 'Connection to 96.9.69.210 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3fa250>, 'Connection to 177.234.7.66 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3fa550>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3fa850>, 'Connection to 115.46.99.97 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3fa050>, 'Connection to 46.150.174.192 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3fa490>, 'Connection to 85.181.3.172 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3faed0>, 'Connection to 46.151.252.82 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
Changing proxy
Takes 660.08775115 seconds
Missing 9 of pills data
sleeping...
waking up!
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 400 Bad Request',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db917350>, 'Connection to 177.128.154.20 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db917750>, 'Connection to 178.217.107.8 timed out. (connect timeout=20)'))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db917950>, 'Connection to 119.42.87.140 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db337790>, 'Connection to 176.15.30.216 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Too many open connections',)))
failed
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db917d10>, 'Connection to 61.7.131.124 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db5dda10>, 'Connection to 47.206.51.67 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db90fb90>, 'Connection to 46.118.119.149 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Takes 718.706940174 seconds
Missing 6 of pills data
sleeping...
waking up!
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb4ab90>, 'Connection to 183.88.65.237 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb4a350>, 'Connection to 213.6.40.142 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry

HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')",),))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4dd690>, 'Connection to 61.7.181.151 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4ddad0>, 'Connection to 113.93.224.54 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 504 Gateway Time-out',)))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7c9390>, 'Connection to 103.225.172.21 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Service Unavailable',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db36ad90>, 'Connection to 196.223.140.170 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db377d90>, 'Connection to 61.6.71.21 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db36a810>, 'Connection to 121.199.42.198 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db36a410>, 'Connection to 103.72.164.25 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db36a290>, 'Connection to 199.127.63.86 timed out. (connect timeout=20)'))
failed
Changing proxy
Takes 798.722761154 seconds
Missing 10 of pills data
sleeping...
waking up!
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 504 Gateway Time-out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4dd5d0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7c9410>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7c99d0>, 'Connection to 91.197.132.99 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db36a210>, 'Connection to 139.224.80.139 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 501 Not Implemented',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db36a410>, 'Connection to 217.150.61.53 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3778d0>, 'Connection to 190.186.58.252 timed out. (connect timeout=20)'))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db46ab50>, 'Connection to 58.48.167.200 timed out. (connect timeout=20)'))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Changing proxy
Retry
Takes 650.80125308 seconds
Missing 8 of pills data
sleeping...
waking up!
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7c9510>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy

('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbaf2bd0>, 'Connection to 121.232.225.252 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db36ad50>, 'Connection to 171.6.72.145 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7c9210>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db46a610>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db46a3d0>, 'Connection to 139.224.80.139 timed out. (connect timeout=20)'))
failed
Changing proxy
Takes 575.864456177 seconds
Missing 7 of pills data
sleeping...
waking up!
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb787d0>, 'Connection to 66.242.91.23 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb78b10>, 'Connection to 207.246.110.99 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db5a0d50>, 'Connection to 91.109.144.217 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4dd8d0>, 'Connection to 122.183.139.107 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4ddb50>, 'Connection to 119.42.85.142 timed out. (connect timeout=20)'))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
Timeout occured
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db36aa50>, 'Connection to 110.77.201.228 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db36a6d0>, 'Connection to 186.223.93.248 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db36aa50>, 'Connection to 110.77.172.23 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db36a5d0>, 'Connection to 103.218.24.76 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db36a090>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db36a310>, 'Connection to 121.199.42.198 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db377dd0>, 'Connection to 45.32.138.138 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db6a1d50>, 'Connection to 176.215.252.152 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db6a1f50>, 'Connection to 175.163.150.18 timed out. (connect timeout=20)'))
failed
Changing proxy
Takes 727.759135008 seconds
Missing 10 of pills data
sleeping...
waking up!
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Service Unavailable',)))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db377650>, 'Connection to 58.48.167.200 timed out. (connect timeout=20)'))
failed
Changing proxy

HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db377f50>, 'Connection to 36.75.244.90 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 504 Gateway Time-out',)))
failed
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Too many open connections',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 504 Gateway Time-out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbafb050>, 'Connection to 1.196.160.149 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbafb550>, 'Connection to 110.77.236.20 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbafbdd0>, 'Connection to 119.42.112.127 timed out. (connect timeout=20)'))
failed
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
Takes 531.226850033 seconds
Missing 8 of pills data
sleeping...
waking up!
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db36aad0>, 'Connection to 82.147.116.201 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db46a990>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
Takes 482.474947929 seconds
Missing 5 of pills data
sleeping...
waking up!
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db96cc90>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db96cb50>, 'Connection to 119.42.85.135 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bf050>, 'Connection to 103.218.24.76 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bf250>, 'Connection to 99.40.66.121 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bf0d0>, 'Connection to 177.128.154.20 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bf410>, 'Connection to 121.129.127.209 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bf490>, 'Connection to 61.7.131.124 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bffd0>, 'Connection to 36.75.244.90 timed out. (connect timeout=20)'))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bc390>, 'Connection to 110.77.236.20 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry

HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bc510>, 'Connection to 139.224.80.139 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bc610>, 'Connection to 91.107.5.210 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 504 Gateway Time-out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bc690>, 'Connection to 176.215.252.152 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Takes 895.747004986 seconds
Missing 14 of pills data
sleeping...
waking up!
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bf290>, 'Connection to 122.183.139.109 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bfcd0>, 'Connection to 83.174.212.197 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bc850>, 'Connection to 66.242.91.23 timed out. (connect timeout=20)'))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bc5d0>, 'Connection to 181.112.61.162 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Service Unavailable',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 504 Gateway Time-out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb38490>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
Timeout occured
Changing proxy
Takes 554.80033493 seconds
Missing 8 of pills data
sleeping...
waking up!
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9cde90>, 'Connection to 85.187.245.138 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')",),))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bc5d0>, 'Connection to 110.78.152.233 timed out. (connect timeout=20)'))
failed
Changing proxy
Takes 357.227425098 seconds
Missing 5 of pills data
sleeping...
waking up!
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bc150>, 'Connection to 177.157.76.247 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bcd50>, 'Connection to 180.183.232.236 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bc550>, 'Connection to 27.44.199.12 timed out. (connect timeout=20)'))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db850b90>, 'Connection to 91.109.144.217 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry

HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db850690>, 'Connection to 114.232.171.61 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9cd090>, 'Connection to 196.223.140.170 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db850690>, 'Connection to 103.225.172.21 timed out. (connect timeout=20)'))
failed
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db850e10>, 'Connection to 122.183.139.104 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4ddcd0>, 'Connection to 199.127.63.86 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
Takes 540.60956502 seconds
Missing 8 of pills data
sleeping...
waking up!
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db96ca10>, 'Connection to 168.0.216.166 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db96ce50>, 'Connection to 103.219.213.189 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db96cc90>, 'Connection to 176.31.174.1 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db96ce10>, 'Connection to 61.7.181.151 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb7e750>, 'Connection to 45.77.166.41 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb7e7d0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9cdf90>, 'Connection to 139.224.80.139 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3e7910>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3e74d0>, 'Connection to 121.199.42.198 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Too many open connections',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
Takes 664.851425886 seconds
Missing 10 of pills data
sleeping...
waking up!
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9cd2d0>, 'Connection to 110.77.172.23 timed out. (connect timeout=20)'))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3e7a10>, 'Connection to 183.51.120.102 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3e7850>, 'Connection to 112.95.204.193 timed out. (connect timeout=20)'))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3e7890>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry

Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3e7250>, 'Connection to 185.112.23.135 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 504 Gateway Time-out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3e7810>, 'Connection to 119.42.85.142 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3e7990>, 'Connection to 122.183.139.107 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3e73d0>, 'Connection to 110.78.172.205 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb7e390>, 'Connection to 203.189.149.81 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb7e4d0>, 'Connection to 119.42.112.127 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb7e510>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 504 Gateway Time-out',)))
failed
Changing proxy
Retry
Takes 557.085579872 seconds
Missing 11 of pills data
sleeping...
waking up!
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb88510>, 'Connection to 171.6.72.145 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb88410>, 'Connection to 114.253.203.162 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9cd150>, 'Connection to 191.102.122.3 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9cd150>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9cdfd0>, 'Connection to 46.118.119.149 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9cdfd0>, 'Connection to 103.74.245.19 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9cdfd0>, 'Connection to 222.137.104.142 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9cdcd0>, 'Connection to 91.197.132.99 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9cd6d0>, 'Connection to 123.163.140.91 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9cd950>, 'Connection to 43.252.18.231 timed out. (connect timeout=20)'))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Too many open connections',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9ba650>, 'Connection to 183.30.197.56 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9ba6d0>, 'Connection to 222.137.104.142 timed out. (connect timeout=20)'))
failed
Changing proxy

HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9baf90>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4d86d0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4d8450>, 'Connection to 99.40.66.121 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4d8410>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')",),))
failed
Changing proxy
Retry
Takes 692.907608032 seconds
Missing 12 of pills data
sleeping...
waking up!
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9cd610>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb885d0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb889d0>, 'Connection to 222.209.77.123 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Too many open connections',)))
failed
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4d8510>, 'Connection to 194.28.50.81 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'SSL23_GET_SERVER_HELLO', 'unknown protocol')],)",),))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4d8a10>, 'Connection to 194.135.90.188 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4d40d0>, 'Connection to 191.102.122.3 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Takes 870.873090029 seconds
Missing 8 of pills data
sleeping...
waking up!
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db45a910>, 'Connection to 113.118.97.5 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bf290>, 'Connection to 109.232.159.71 timed out. (connect timeout=20)'))
failed
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bf610>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bfad0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bf3d0>, 'Connection to 177.126.81.63 timed out. (connect timeout=20)'))
failed
Changing proxy

HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bf190>, 'Connection to 46.250.1.75 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Too many open connections',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb60350>, 'Connection to 173.249.9.82 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb60290>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9cd110>, 'Connection to 122.183.139.101 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db81cf50>, 'Connection to 83.174.212.197 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bea90>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9be150>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bed50>, 'Connection to 197.149.94.42 timed out. (connect timeout=20)'))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4d8810>, 'Connection to 163.125.64.248 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9be610>, 'Connection to 139.201.23.82 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9beb50>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bed50>, 'Connection to 176.15.30.216 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db77abd0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db77a850>, 'Connection to 178.124.85.54 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Takes 793.660373926 seconds
Missing 15 of pills data
sleeping...
waking up!
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb60410>, 'Connection to 119.42.85.135 timed out. (connect timeout=20)'))
failed
Changing proxy
Timeout occured
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db96c390>, 'Connection to 36.72.70.175 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db96c690>, 'Connection to 115.58.130.111 timed out. (connect timeout=20)'))
failed
Changing proxy

HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db96ce50>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db81c3d0>, 'Connection to 116.20.229.231 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb88f50>, 'Connection to 82.179.134.236 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db96cc10>, 'Connection to 209.52.241.147 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb88a90>, 'Connection to 62.201.225.42 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db81c610>, 'Connection to 118.179.60.110 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db81ce90>, 'Connection to 203.189.149.81 timed out. (connect timeout=20)'))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9be890>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Internal Server Error',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4d8190>, 'Connection to 181.112.61.162 timed out. (connect timeout=20)'))
failed
Changing proxy
Takes 926.579578876 seconds
Missing 11 of pills data
sleeping...
waking up!
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db96c590>, 'Connection to 37.29.58.228 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bed50>, 'Connection to 180.183.232.236 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9be850>, 'Connection to 72.35.248.249 timed out. (connect timeout=20)'))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4d8490>, 'Connection to 139.224.80.139 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4d87d0>, 'Connection to 103.228.119.125 timed out. (connect timeout=20)'))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4d82d0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Takes 551.001135111 seconds
Missing 7 of pills data
sleeping...
waking up!
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db45ab90>, 'Connection to 221.4.133.67 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db45a690>, 'Connection to 46.16.226.222 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Changing proxy
Retry

Changing proxy
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4d8290>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4d8f50>, 'Connection to 171.37.209.166 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4d8ad0>, 'Connection to 116.20.229.231 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 403 Forbidden',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bebd0>, 'Connection to 78.155.193.226 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bee50>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bec50>, 'Connection to 190.27.196.174 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4d8490>, 'Connection to 218.20.218.200 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4d81d0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4d8f90>, 'Connection to 178.170.185.171 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4d8490>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
Takes 705.985682011 seconds
Missing 10 of pills data
sleeping...
waking up!
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db81c350>, 'Connection to 137.74.254.242 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db81c890>, 'Connection to 188.115.181.216 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4d8110>, 'Connection to 113.77.240.214 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4d8110>, 'Connection to 183.33.192.21 timed out. (connect timeout=20)'))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9be310>, 'Connection to 122.183.139.107 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bead0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bee10>, 'Connection to 41.76.150.154 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9beb50>, 'Connection to 85.187.245.138 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb44d10>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb3a590>, 'Connection to 188.244.6.90 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 504 Gateway Time-out',)))
failed
Changing proxy
Retry

Takes 480.065876007 seconds
Missing 9 of pills data
sleeping...
waking up!
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9cd750>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9cd950>, 'Connection to 94.136.138.142 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9cdfd0>, 'Connection to 139.224.80.139 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9cddd0>, 'Connection to 36.72.70.175 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
Timeout occured
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db81c250>, 'Connection to 37.29.58.228 timed out. (connect timeout=20)'))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb88450>, 'Connection to 178.124.85.54 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb88050>, 'Connection to 191.102.122.3 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb7a990>, 'Connection to 123.114.62.56 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb88810>, 'Connection to 138.185.153.92 timed out. (connect timeout=20)'))
failed
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb88a90>, 'Connection to 1.20.169.26 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb885d0>, 'Connection to 47.206.51.67 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Takes 757.337599993 seconds
Missing 11 of pills data
sleeping...
waking up!
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 502 Proxy Error (  )',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bf110>, 'Connection to 62.201.225.42 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bfe10>, 'Connection to 177.126.81.63 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bf490>, 'Connection to 36.80.184.133 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bf8d0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb894d0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bf910>, 'Connection to 178.170.185.171 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bff90>, 'Connection to 82.179.134.236 timed out. (connect timeout=20)'))
failed
Changing proxy

HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb88d50>, 'Connection to 78.155.193.226 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb7a210>, 'Connection to 189.204.116.221 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb7a310>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb44ed0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb7a9d0>, 'Connection to 119.129.99.134 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Takes 1093.90210605 seconds
Missing 8 of pills data
sleeping...
waking up!
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb60dd0>, 'Connection to 173.249.9.82 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb4a650>, 'Connection to 212.56.204.6 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb4a250>, 'Connection to 159.192.227.221 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Timeout occured
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db81cfd0>, 'Connection to 92.255.186.41 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb4af10>, 'Connection to 204.12.155.204 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb4a090>, 'Connection to 118.179.60.110 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 504 Gateway Time-out',)))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Too many open connections',)))
failed
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bf8d0>, 'Connection to 37.29.82.115 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bf550>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bfcd0>, 'Connection to 188.115.181.216 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb7a2d0>, 'Connection to 113.118.97.5 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb3ad90>, 'Connection to 119.129.99.134 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Takes 1094.78827405 seconds
Missing 11 of pills data
sleeping...
waking up!
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb8cb10>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy

HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb8cd10>, 'Connection to 121.199.42.198 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb8cad0>, 'Connection to 139.224.80.139 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb8c310>, 'Connection to 61.6.50.207 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bf6d0>, 'Connection to 190.27.196.174 timed out. (connect timeout=20)'))
failed
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bff90>, 'Connection to 109.232.159.71 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bf0d0>, 'Connection to 112.95.204.193 timed out. (connect timeout=20)'))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bfed0>, 'Connection to 91.107.5.210 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bf790>, 'Connection to 194.135.90.188 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Takes 651.285462141 seconds
Missing 9 of pills data
sleeping...
waking up!
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Too many open connections',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 501 Not Implemented',)))
failed
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb44f10>, 'Connection to 185.85.21.6 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb7a390>, 'Connection to 122.183.139.101 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb7a950>, 'Connection to 113.116.159.20 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb7a190>, 'Connection to 91.222.215.25 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bfa90>, 'Connection to 46.250.8.234 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bf110>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bf6d0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bfb90>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bfdd0>, 'Connection to 60.214.155.243 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry

HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bf750>, 'Connection to 176.15.30.216 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bf590>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 504 Gateway Time-out',)))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb7afd0>, 'Connection to 113.77.240.214 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db81ce10>, 'Connection to 41.76.150.154 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db81cc10>, 'Connection to 222.209.77.123 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db81c5d0>, 'Connection to 183.17.123.107 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb8c9d0>, 'Connection to 122.189.199.126 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb8c690>, 'Connection to 221.204.117.250 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
Takes 930.387739897 seconds
Missing 15 of pills data
sleeping...
waking up!
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb896d0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb89550>, 'Connection to 81.36.23.100 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3adcd0>, 'Connection to 81.36.23.100 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3adb90>, 'Connection to 121.199.42.198 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3adc50>, 'Connection to 179.125.127.90 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3bb150>, 'Connection to 181.112.152.222 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3bb550>, 'Connection to 61.6.22.99 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db32e350>, 'Connection to 110.77.212.225 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3bbe50>, 'Connection to 61.7.178.221 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3dae10>, 'Connection to 62.32.75.82 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3dadd0>, 'Connection to 31.173.209.111 timed out. (connect timeout=20)'))
failed
Changing proxy

HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Takes 650.679488182 seconds
Missing 10 of pills data
sleeping...
waking up!
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb89850>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3bbb90>, 'Connection to 91.229.222.163 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb89510>, 'Connection to 92.255.186.41 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3adc50>, 'Connection to 82.179.134.240 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3adcd0>, 'Connection to 209.52.241.147 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3bb9d0>, 'Connection to 41.76.150.154 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 504 Gateway Time-out',)))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3bb090>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3d3f90>, 'Connection to 59.38.63.193 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3da4d0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
Changing proxy
Takes 549.00058198 seconds
Missing 10 of pills data
sleeping...
waking up!
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb899d0>, 'Connection to 182.253.177.80 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Too many open connections',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb89310>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3bb810>, 'Connection to 178.217.107.8 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3d3690>, 'Connection to 113.118.97.5 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3daf10>, 'Connection to 91.192.2.168 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry

Changing proxy
Takes 589.746273041 seconds
Missing 7 of pills data
sleeping...
waking up!
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb7ad50>, 'Connection to 194.28.50.81 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb7a9d0>, 'Connection to 87.226.195.37 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb7ac10>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb7ab90>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'SSL23_GET_SERVER_HELLO', 'unknown protocol')],)",),))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb4a210>, 'Connection to 91.222.215.25 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb4a190>, 'Connection to 110.77.208.180 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bf950>, 'Connection to 78.25.98.210 timed out. (connect timeout=20)'))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Too many open connections',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb4aed0>, 'Connection to 123.138.89.133 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
Timeout occured
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bfbd0>, 'Connection to 61.6.10.138 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bad10>, 'Connection to 62.113.46.235 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9ba510>, 'Connection to 61.7.181.88 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Too many open connections',)))
failed
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db406d90>, 'Connection to 195.209.125.98 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db406f50>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
Takes 602.40202117 seconds
Missing 13 of pills data
sleeping...
waking up!
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bac10>, 'Connection to 212.231.65.8 timed out. (connect timeout=20)'))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb7e190>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9a5610>, 'Connection to 115.61.99.254 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbfecb50>, 'Connection to 59.38.63.193 timed out. (connect timeout=20)'))
failed
Changing proxy

HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbaebf50>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbaeb0d0>, 'Connection to 145.255.28.218 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbaebf50>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 504 Gateway Time-out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbfecfd0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 504 Gateway Time-out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Too many open connections',)))
failed
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbfecdd0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbfece90>, 'Connection to 115.61.99.254 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 500 Internal Server Error',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db6249d0>, 'Connection to 91.220.38.251 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbfeca10>, 'Connection to 178.22.248.124 timed out. (connect timeout=20)'))
failed
Changing proxy
Takes 472.959967136 seconds
Missing 11 of pills data
sleeping...
waking up!
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb7e650>, 'Connection to 193.106.25.33 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bab90>, 'Connection to 189.90.96.5 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9baa10>, 'Connection to 217.29.28.183 timed out. (connect timeout=20)'))
failed
Changing proxy
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbfec910>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbaf9150>, 'Connection to 191.102.122.3 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbaf9ad0>, 'Connection to 176.106.216.16 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db663490>, 'Connection to 46.250.8.234 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db663990>, 'Connection to 159.192.227.221 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 504 Gateway Time-out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb7e110>, 'Connection to 116.106.4.132 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db663510>, 'Connection to 201.72.43.195 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry

HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db663f10>, 'Connection to 81.88.198.162 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db663450>, 'Connection to 186.67.90.13 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db6639d0>, 'Connection to 113.118.97.5 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Takes 600.477250099 seconds
Missing 9 of pills data
sleeping...
waking up!
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bac10>, 'Connection to 105.22.41.98 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9baa90>, 'Connection to 194.126.182.30 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9ba590>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bac10>, 'Connection to 5.197.231.174 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db5a5b90>, 'Connection to 177.128.150.22 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db5956d0>, 'Connection to 182.253.177.80 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db5955d0>, 'Connection to 91.220.38.251 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db5a4810>, 'Connection to 61.6.22.99 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3c8a50>, 'Connection to 81.36.23.100 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db5a4ad0>, 'Connection to 103.28.226.92 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db5a4b10>, 'Connection to 113.116.125.178 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db5a4c50>, 'Connection to 122.183.139.107 timed out. (connect timeout=20)'))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 504 Gateway Time-out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db5a4b90>, 'Connection to 103.205.177.44 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db5a4ad0>, 'Connection to 116.106.4.132 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db595a50>, 'Connection to 121.199.42.198 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db5a4c50>, 'Connection to 159.192.227.221 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 504 Gateway Time-out',)))
failed
Changing proxy
Changing proxy
Retry

HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Too many open connections',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db5a49d0>, 'Connection to 82.179.134.240 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Takes 797.370155096 seconds
Missing 13 of pills data
sleeping...
waking up!
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9ba150>, 'Connection to 145.255.28.218 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Too many open connections',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9ba990>, 'Connection to 31.173.209.111 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9ba210>, 'Connection to 91.192.2.168 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
Timeout occured
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db78d210>, 'Connection to 47.206.51.67 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db581190>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7872d0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db787910>, 'Connection to 194.28.50.32 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Takes 590.367908955 seconds
Missing 7 of pills data
sleeping...
waking up!
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9ba810>, 'Connection to 182.121.203.252 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9babd0>, 'Connection to 13.78.125.167 timed out. (connect timeout=20)'))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db581fd0>, 'Connection to 116.58.227.159 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db59db90>, 'Connection to 81.88.198.162 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db59d2d0>, 'Connection to 103.90.200.2 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db59dcd0>, 'Connection to 201.72.43.195 timed out. (connect timeout=20)'))
failed
Changing proxy
Takes 404.057106018 seconds
Missing 6 of pills data
sleeping...
waking up!
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb44f10>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bf650>, 'Connection to 113.116.125.178 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Too many open connections',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bf490>, 'Connection to 112.95.204.193 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')",),))
failed
Changing proxy
Retry

HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Too many open connections',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bfcd0>, 'Connection to 116.58.227.64 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bf6d0>, 'Connection to 110.77.212.225 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 504 Gateway Time-out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bfc10>, 'Connection to 197.255.45.6 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bf8d0>, 'Connection to 196.223.140.170 timed out. (connect timeout=20)'))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bf6d0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 501 Not Implemented',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bf050>, 'Connection to 212.231.65.8 timed out. (connect timeout=20)'))
failed
Changing proxy
Takes 461.737262964 seconds
Missing 11 of pills data
sleeping...
waking up!
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7c9550>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7c9a10>, 'Connection to 5.197.231.174 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 500 Internal Server Error',)))
failed
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db59ef90>, 'Connection to 122.183.139.109 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7c9bd0>, 'Connection to 139.224.80.139 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7c9150>, 'Connection to 61.7.181.88 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb44110>, 'Connection to 78.25.98.210 timed out. (connect timeout=20)'))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Too many open connections',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Service Unavailable',)))
failed
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db59d7d0>, 'Connection to 62.113.46.235 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db59dc10>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db59d7d0>, 'Connection to 189.90.96.5 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry

Takes 500.48725605 seconds
Missing 11 of pills data
sleeping...
waking up!
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb44f10>, 'Connection to 194.126.182.30 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Too many open connections',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb44110>, 'Connection to 183.51.191.135 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db59e2d0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db59efd0>, 'Connection to 139.224.80.139 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db59e2d0>, 'Connection to 121.199.42.198 timed out. (connect timeout=20)'))
failed
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 500 Internal Privoxy Error',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Too many open connections',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db78d810>, 'Connection to 181.112.152.222 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db78dd90>, 'Connection to 113.118.97.5 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bafd0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb7a510>, 'Connection to 51.15.137.244 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb7a590>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Takes 578.229040861 seconds
Missing 10 of pills data
sleeping...
waking up!
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db5a4b50>, 'Connection to 193.106.25.33 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb44b10>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb44e10>, 'Connection to 186.67.90.13 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb44110>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb44b10>, 'Connection to 41.76.150.154 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb441d0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb44d90>, 'Connection to 171.8.170.134 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb44190>, 'Connection to 188.254.78.108 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb44390>, 'Connection to 187.108.38.214 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry

HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb440d0>, 'Connection to 1.196.160.149 timed out. (connect timeout=20)'))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db78d950>, 'Connection to 61.7.178.221 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db59e790>, 'Connection to 176.106.216.16 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db59e410>, 'Connection to 62.32.75.82 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7c5110>, 'Connection to 50.93.202.82 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db59de90>, 'Connection to 182.253.177.80 timed out. (connect timeout=20)'))
failed
Changing proxy
Takes 740.18483901 seconds
Missing 12 of pills data
sleeping...
waking up!
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9792d0>, 'Connection to 194.28.50.81 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db979d10>, 'Connection to 5.197.231.174 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db979510>, 'Connection to 179.125.127.90 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9792d0>, 'Connection to 193.106.25.33 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 504 Gateway Time-out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9ba4d0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db979150>, 'Connection to 13.78.125.167 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Service Unavailable',)))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Too many open connections',)))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb06550>, 'Connection to 181.112.152.222 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb06c10>, 'Connection to 186.67.90.13 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb067d0>, 'Connection to 91.229.222.163 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb06450>, 'Connection to 116.106.4.132 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb06710>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb06a10>, 'Connection to 122.183.139.107 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry

HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb06550>, 'Connection to 177.128.150.22 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7c51d0>, 'Connection to 189.90.96.5 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4cffd0>, 'Connection to 176.106.39.20 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7c5e10>, 'Connection to 122.183.139.98 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7c5b50>, 'Connection to 62.113.46.235 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7c5290>, 'Connection to 176.106.216.16 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7c5750>, 'Connection to 139.224.80.139 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7ac190>, 'Connection to 194.28.50.80 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Takes 814.702574015 seconds
Missing 14 of pills data
sleeping...
waking up!
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bf090>, 'Connection to 103.90.200.2 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bf490>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db595a90>, 'Connection to 61.7.181.88 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db595e50>, 'Connection to 116.58.227.64 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb06090>, 'Connection to 139.224.80.139 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb06b50>, 'Connection to 196.223.140.170 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb06990>, 'Connection to 49.205.250.0 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb06450>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 500 Internal Server Error',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb06590>, 'Connection to 62.32.75.82 timed out. (connect timeout=20)'))
failed
Changing proxy

Takes 573.211228132 seconds
Missing 11 of pills data
sleeping...
waking up!
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb441d0>, 'Connection to 197.255.45.6 timed out. (connect timeout=20)'))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db5956d0>, 'Connection to 187.108.38.214 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Too many open connections',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db46add0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4cfb50>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
Takes 755.365036011 seconds
Missing 6 of pills data
sleeping...
waking up!
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db406c90>, 'Connection to 51.15.137.244 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7bde50>, 'Connection to 176.15.163.199 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db595050>, 'Connection to 36.66.87.18 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Too many open connections',)))
failed
Changing proxy
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 504 Gateway Time-out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bf510>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bf5d0>, 'Connection to 121.199.42.198 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bfa50>, 'Connection to 78.25.98.210 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9bfe50>, 'Connection to 186.83.66.119 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Retry
Timeout occured
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4b4650>, 'Connection to 27.40.144.177 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4b4a50>, 'Connection to 212.231.65.8 timed out. (connect timeout=20)'))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4b4d10>, 'Connection to 188.254.78.108 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 500 Internal Privoxy Error',)))
failed
Changing proxy
Retry

Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4cf2d0>, 'Connection to 121.199.42.198 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Takes 676.256441116 seconds
Missing 11 of pills data
sleeping...
waking up!
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db406310>, 'Connection to 191.102.122.3 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db406a50>, 'Connection to 103.205.177.44 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7de390>, 'Connection to 31.25.141.46 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Too many open connections',)))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4b4950>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Too many open connections',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Too many open connections',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4b4350>, 'Connection to 105.22.41.98 timed out. (connect timeout=20)'))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4b4190>, 'Connection to 82.179.134.240 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4b4d50>, 'Connection to 116.58.227.159 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb068d0>, 'Connection to 194.28.50.32 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb06650>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7c5310>, 'Connection to 110.77.212.225 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb06090>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
Takes 569.501539946 seconds
Missing 11 of pills data
sleeping...
waking up!
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb44410>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb44c50>, 'Connection to 61.7.178.221 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9794d0>, 'Connection to 183.30.197.27 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4066d0>, 'Connection to 115.225.206.161 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4063d0>, 'Connection to 103.28.226.92 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry

('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4068d0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db595b10>, 'Connection to 122.183.137.190 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db72ba10>, 'Connection to 222.209.77.123 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db742250>, 'Connection to 194.28.50.80 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Too many open connections',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 500 Internal Privoxy Error',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db742d10>, 'Connection to 186.83.66.119 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')",),))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db731650>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Takes 642.051307917 seconds
Missing 9 of pills data
sleeping...
waking up!
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db59e710>, 'Connection to 194.28.50.81 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7c9790>, 'Connection to 188.115.187.207 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb4ad90>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb4aed0>, 'Connection to 201.72.43.195 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db78df90>, 'Connection to 177.22.81.102 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db979f90>, 'Connection to 14.117.208.114 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb44550>, 'Connection to 42.94.164.240 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db78dd90>, 'Connection to 119.123.179.214 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb4a890>, 'Connection to 188.165.240.92 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db406690>, 'Connection to 50.93.202.82 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4068d0>, 'Connection to 200.13.160.125 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4062d0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db406b50>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db75b1d0>, 'Connection to 81.22.54.60 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry

HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Too many open connections',)))
failed
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db406f10>, 'Connection to 221.204.117.250 timed out. (connect timeout=20)'))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db72bc90>, 'Connection to 91.220.38.251 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db72bd10>, 'Connection to 46.118.119.149 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db72b750>, 'Connection to 95.182.108.179 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db72b650>, 'Connection to 177.128.159.173 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db72bad0>, 'Connection to 13.78.125.167 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db731950>, 'Connection to 60.24.170.249 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Retry
Takes 855.179248095 seconds
Missing 17 of pills data
sleeping...
waking up!
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db752c10>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db731c50>, 'Connection to 109.167.129.14 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db731c50>, 'Connection to 113.92.93.18 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db6ad510>, 'Connection to 122.183.139.109 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db6ad510>, 'Connection to 183.30.197.27 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db6ad350>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db6ad550>, 'Connection to 203.128.68.242 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Takes 624.317448139 seconds
Missing 7 of pills data
sleeping...
waking up!
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db75b710>, 'Connection to 61.6.87.121 timed out. (connect timeout=20)'))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 501 Not Implemented',)))
failed
Changing proxy
Retry
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbaeb110>, 'Connection to 119.129.99.134 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbaeb250>, 'Connection to 163.125.196.4 timed out. (connect timeout=20)'))
failed
Changing proxy

HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbaeb390>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbaeb450>, 'Connection to 196.223.140.170 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Too many open connections',)))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb7e110>, 'Connection to 113.93.224.54 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbaf9e90>, 'Connection to 139.224.80.139 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbaf9950>, 'Connection to 103.90.200.2 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Takes 564.456673145 seconds
Missing 9 of pills data
sleeping...
waking up!
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db75b710>, 'Connection to 125.89.53.250 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db75b990>, 'Connection to 41.76.150.154 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Too many open connections',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db75bf90>, 'Connection to 59.38.60.132 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db75b1d0>, 'Connection to 197.255.45.6 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db75b250>, 'Connection to 110.77.236.203 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb7e810>, 'Connection to 113.227.241.208 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db75b690>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db72ee90>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
Timeout occured
Changing proxy
Changing proxy
Retry
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db72e890>, 'Connection to 122.183.139.104 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 504 Gateway Time-out',)))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbaf9850>, 'Connection to 49.205.250.0 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry

HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbaf9390>, 'Connection to 122.183.139.101 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbaf9490>, 'Connection to 113.104.196.29 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbaf9550>, 'Connection to 91.107.5.210 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbaf9a10>, 'Connection to 103.28.226.92 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Takes 852.19445014 seconds
Missing 16 of pills data
sleeping...
waking up!
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db5a5f10>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db5a5f50>, 'Connection to 78.9.110.69 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db5a5d90>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db5a5790>, 'Connection to 121.129.127.209 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Service Unavailable',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 403 Forbidden',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db752750>, 'Connection to 123.138.89.133 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 504 Gateway Time-out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dba56b10>, 'Connection to 121.199.42.198 timed out. (connect timeout=20)'))
failed
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db96a510>, 'Connection to 83.170.214.90 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Takes 618.221991062 seconds
Missing 9 of pills data
sleeping...
waking up!
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbfecf90>, 'Connection to 88.147.159.167 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbfec910>, 'Connection to 60.24.170.249 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
Timeout occured
Changing proxy
Retry
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db6ab590>, 'Connection to 118.254.142.42 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry

HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dba495d0>, 'Connection to 109.167.205.118 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dba49190>, 'Connection to 91.234.183.15 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Timeout occured
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dba49d90>, 'Connection to 194.126.182.30 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dba495d0>, 'Connection to 82.147.116.201 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dba49f10>, 'Connection to 111.176.182.218 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dba49c10>, 'Connection to 110.77.236.203 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dba49850>, 'Connection to 186.83.66.119 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dba497d0>, 'Connection to 46.151.248.167 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dba49f50>, 'Connection to 139.224.80.139 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Takes 934.705229044 seconds
Missing 15 of pills data
sleeping...
waking up!
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db731590>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db731550>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db731b50>, 'Connection to 221.204.117.250 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db6ab410>, 'Connection to 123.138.89.133 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db6ab490>, 'Connection to 78.9.110.69 timed out. (connect timeout=20)'))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Service Unavailable',)))
failed
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb97f90>, 'Connection to 42.104.84.106 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry

('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb97710>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Too many open connections',)))
failed
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db72ee10>, 'Connection to 41.76.150.154 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Takes 535.753021002 seconds
Missing 8 of pills data
sleeping...
waking up!
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db406ad0>, 'Connection to 122.183.139.101 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb7e450>, 'Connection to 188.165.240.92 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db6ad710>, 'Connection to 82.102.14.190 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db6ad9d0>, 'Connection to 112.86.73.52 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db72e3d0>, 'Connection to 111.176.182.218 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db72ee90>, 'Connection to 115.58.130.111 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb97a10>, 'Connection to 122.183.137.190 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb97a50>, 'Connection to 101.99.80.135 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb97210>, 'Connection to 145.255.28.218 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db731810>, 'Connection to 1.20.169.26 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db731410>, 'Connection to 109.72.234.147 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db731d90>, 'Connection to 177.22.81.102 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Service Unavailable',)))
failed
Changing proxy
Retry
Takes 741.879808903 seconds
Missing 9 of pills data
sleeping...
waking up!
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbfecc50>, 'Connection to 45.32.112.136 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db72ecd0>, 'Connection to 119.39.68.143 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db72e150>, 'Connection to 188.120.209.97 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db98ed10>, 'Connection to 51.15.137.244 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db98ee50>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Retry

HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db72e190>, 'Connection to 103.105.212.58 timed out. (connect timeout=20)'))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db98e150>, 'Connection to 112.86.73.52 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db973950>, 'Connection to 42.94.164.240 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Too many open connections',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db973a50>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 504 Gateway Time-out',)))
failed
Changing proxy
Retry
Takes 652.536885977 seconds
Missing 10 of pills data
sleeping...
waking up!
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db973790>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db973ed0>, 'Connection to 47.206.51.67 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Too many open connections',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db731290>, 'Connection to 91.107.5.210 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db731190>, 'Connection to 115.58.130.111 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 500 Internal Server Error',)))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db98ee90>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db98ea50>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db98e050>, 'Connection to 42.94.164.240 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Too many open connections',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db98e610>, 'Connection to 201.72.43.195 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Takes 533.710093975 seconds
Missing 8 of pills data
sleeping...
waking up!
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb9d1d0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db731d50>, 'Connection to 195.133.220.18 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry

HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db731d90>, 'Connection to 121.237.137.225 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7319d0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db731290>, 'Connection to 183.51.191.135 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db414750>, 'Connection to 103.251.164.12 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db414690>, 'Connection to 84.204.144.174 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db414250>, 'Connection to 121.199.42.198 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Takes 595.723903894 seconds
Missing 9 of pills data
sleeping...
waking up!
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbaf9b90>, 'Connection to 122.183.139.101 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbaf9390>, 'Connection to 218.73.130.151 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbaf9810>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbaf9150>, 'Connection to 116.212.140.13 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbaf9890>, 'Connection to 45.32.112.136 timed out. (connect timeout=20)'))
failed
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbaf9950>, 'Connection to 46.151.248.167 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbaf9dd0>, 'Connection to 212.56.204.6 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db75bfd0>, 'Connection to 183.30.197.27 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db75b210>, 'Connection to 121.199.42.198 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db75b390>, 'Connection to 51.15.137.244 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db75bf10>, 'Connection to 110.77.210.95 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db6ad4d0>, 'Connection to 50.93.200.253 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db6adad0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db6ad850>, 'Connection to 186.67.90.13 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy

Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Too many open connections',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db406190>, 'Connection to 164.160.142.60 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by SSLError(SSLError("bad handshake: SysCallError(104, 'ECONNRESET')",),))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4062d0>, 'Connection to 42.104.84.106 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Too many open connections',)))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db98fb50>, 'Connection to 121.31.176.56 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db406450>, 'Connection to 58.48.167.200 timed out. (connect timeout=20)'))
failed
Changing proxy
Takes 802.649246931 seconds
Missing 16 of pills data
sleeping...
waking up!
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db5a5350>, 'Connection to 117.95.83.26 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 500 Internal Server Error',)))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb780d0>, 'Connection to 1.194.129.37 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7d2dd0>, 'Connection to 1.194.129.37 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7d2b50>, 'Connection to 137.74.254.242 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db40a910>, 'Connection to 122.183.139.107 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Takes 582.735528946 seconds
Missing 5 of pills data
sleeping...
waking up!
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db72eed0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7d2fd0>, 'Connection to 218.73.131.95 timed out. (connect timeout=20)'))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7d2850>, 'Connection to 111.176.182.218 timed out. (connect timeout=20)'))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7d2690>, 'Connection to 110.77.215.68 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Takes 410.651916027 seconds
Missing 5 of pills data
sleeping...
waking up!
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbaeb3d0>, 'Connection to 36.78.187.120 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbaeb0d0>, 'Connection to 117.95.83.26 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbaebb90>, 'Connection to 190.152.4.54 timed out. (connect timeout=20)'))
failed
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry

Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 504 Gateway Time-out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db406890>, 'Connection to 139.224.80.139 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db75b5d0>, 'Connection to 143.202.208.151 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db402610>, 'Connection to 137.74.168.174 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db5a57d0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db98f350>, 'Connection to 190.186.58.252 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db98f6d0>, 'Connection to 182.121.203.252 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Takes 588.284958839 seconds
Missing 10 of pills data
sleeping...
waking up!
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db406690>, 'Connection to 103.219.213.35 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db973b10>, 'Connection to 185.65.195.6 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb78a90>, 'Connection to 139.224.80.139 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 504 Gateway Time-out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db973710>, 'Connection to 58.48.167.200 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Too many open connections',)))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db98fa90>, 'Connection to 180.241.167.230 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db98fa50>, 'Connection to 115.221.127.8 timed out. (connect timeout=20)'))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db98f550>, 'Connection to 186.194.183.9 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db98f490>, 'Connection to 88.119.49.66 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db411a90>, 'Connection to 5.197.231.174 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Takes 660.978531122 seconds
Missing 9 of pills data
sleeping...
waking up!
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db973ed0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4025d0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db402150>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4020d0>, 'Connection to 195.133.220.18 timed out. (connect timeout=20)'))
failed
Changing proxy

HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db402890>, 'Connection to 122.183.139.98 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db98f310>, 'Connection to 195.208.128.117 timed out. (connect timeout=20)'))
failed
Changing proxy
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7d2890>, 'Connection to 95.182.109.147 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 504 Gateway Time-out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7d2a50>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 504 Gateway Time-out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db414050>, 'Connection to 122.183.139.104 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db98f3d0>, 'Connection to 61.6.119.124 timed out. (connect timeout=20)'))
failed
Changing proxy
Takes 1036.11643004 seconds
Missing 8 of pills data
sleeping...
waking up!
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db4028d0>, 'Connection to 46.118.119.149 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db402510>, 'Connection to 171.88.53.199 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7d20d0>, 'Connection to 115.221.127.8 timed out. (connect timeout=20)'))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7d26d0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7d2990>, 'Connection to 41.76.150.154 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7d2810>, 'Connection to 113.118.97.5 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db899210>, 'Connection to 223.204.194.1 timed out. (connect timeout=20)'))
failed
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db899190>, 'Connection to 177.67.216.166 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3c7150>, 'Connection to 173.249.9.82 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3c7150>, 'Connection to 183.51.191.135 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3c7050>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db8e01d0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Service Unavailable',)))
failed
Changing proxy
Retry

HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3c7550>, 'Connection to 190.152.4.54 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Too many open connections',)))
failed
Changing proxy
Retry
Takes 607.876008987 seconds
Missing 10 of pills data
sleeping...
waking up!
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db752510>, 'Connection to 60.167.133.76 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7525d0>, 'Connection to 181.65.131.102 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db5a50d0>, 'Connection to 91.210.94.182 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db5a5250>, 'Connection to 164.160.142.60 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb78190>, 'Connection to 5.197.231.174 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db402910>, 'Connection to 91.193.130.172 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db88be90>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3a91d0>, 'Connection to 125.209.86.212 timed out. (connect timeout=20)'))
failed
Changing proxy
Timeout occured
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Takes 804.129981995 seconds
Missing 9 of pills data
sleeping...
waking up!
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db8990d0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db899c90>, 'Connection to 201.72.43.195 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db899550>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db663650>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db663150>, 'Connection to 103.74.245.25 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db663710>, 'Connection to 61.6.119.124 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db5a5850>, 'Connection to 49.70.64.29 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db5a5d90>, 'Connection to 116.212.140.13 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7c96d0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy

HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7c9a10>, 'Connection to 180.241.167.230 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7c9150>, 'Connection to 110.77.210.95 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7c94d0>, 'Connection to 122.183.139.101 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Retry
Takes 596.525427818 seconds
Missing 9 of pills data
sleeping...
waking up!
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db899250>, 'Connection to 83.170.214.90 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db88b1d0>, 'Connection to 177.207.193.218 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db88ba90>, 'Connection to 121.199.42.198 timed out. (connect timeout=20)'))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Timeout occured
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db36a490>, 'Connection to 190.186.58.252 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3b1510>, 'Connection to 181.112.221.182 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3b19d0>, 'Connection to 103.205.177.44 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3b1050>, 'Connection to 106.56.102.237 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3a9050>, 'Connection to 45.32.112.136 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3a9990>, 'Connection to 112.115.29.36 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3a9810>, 'Connection to 41.76.150.154 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3a9f90>, 'Connection to 171.97.81.129 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3a9990>, 'Connection to 61.7.177.49 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3a9090>, 'Connection to 94.75.134.70 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3a9cd0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 504 Gateway Time-out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7c90d0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry

HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3a9e90>, 'Connection to 110.77.215.68 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Takes 731.539469957 seconds
Missing 13 of pills data
sleeping...
waking up!
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db8e5f10>, 'Connection to 176.215.249.10 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3b6050>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3b6750>, 'Connection to 104.199.192.122 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7c9a50>, 'Connection to 143.202.208.151 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db8e0f50>, 'Connection to 179.111.212.62 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db8e0f10>, 'Connection to 89.28.253.232 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db8e0ad0>, 'Connection to 49.70.64.29 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db8e0f10>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7c9850>, 'Connection to 123.161.153.134 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7c9350>, 'Connection to 195.208.128.117 timed out. (connect timeout=20)'))
failed
Changing proxy
Timeout occured
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db8508d0>, 'Connection to 42.104.84.106 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db46a690>, 'Connection to 47.206.51.67 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Takes 756.150867939 seconds
Missing 8 of pills data
sleeping...
waking up!
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db8e5650>, 'Connection to 115.146.126.223 timed out. (connect timeout=20)'))
failed
Changing proxy
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3a9210>, 'Connection to 191.102.122.3 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db8e0fd0>, 'Connection to 200.13.160.125 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 504 Gateway Time-out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db8e0090>, 'Connection to 195.22.125.66 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db8e0050>, 'Connection to 123.161.153.134 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3b6f10>, 'Connection to 177.22.81.102 timed out. (connect timeout=20)'))
failed
Changing proxy

Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3b6a50>, 'Connection to 37.29.82.115 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3b66d0>, 'Connection to 186.67.90.13 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3b6810>, 'Connection to 183.51.191.135 timed out. (connect timeout=20)'))
failed
Changing proxy
Takes 490.799429893 seconds
Missing 8 of pills data
sleeping...
waking up!
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 504 Gateway Time-out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db8e5b50>, 'Connection to 138.0.204.122 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 504 Gateway Time-out',)))
failed
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3a95d0>, 'Connection to 194.28.50.32 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3a9a10>, 'Connection to 187.12.121.138 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 504 Gateway Time-out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3a9a10>, 'Connection to 112.115.29.36 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3a9d10>, 'Connection to 122.183.137.190 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db3a9090>, 'Connection to 139.224.80.139 timed out. (connect timeout=20)'))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9b9710>, 'Connection to 123.138.89.133 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Timeout occured
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9b9cd0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7872d0>, 'Connection to 123.207.25.143 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Takes 508.601205826 seconds
Missing 10 of pills data
sleeping...
waking up!
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
        carTypeName = None
    startTime = user_act['chargeTime']
    #AI detect startTime
    if len(startTime) < 6:
    #if in the expression of hours:
        if hour_uni in startTime:
            idx_hour = len(startTime)-3
            hours = int(startTime[:idx_hour])
            startTime = startTime = datetime.datetime.now() - datetime.timedelta(hours=hours)
#                     startTime = '{:%Y-%m-%d %H:%M:%S}'.format(startTime)+"-07"
            startTime = '{:%Y-%m-%d %H:%M}'.format(startTime)+"-07"
    #if in the expression of minutes:
        if minute_uni in startTime:
            idx_min = len(startTime)-3
            minutes = int(startTime[:idx_min])
            startTime = startTime = datetime.datetime.now() - datetime.timedelta(minutes=minutes)
#                     startTime = '{:%Y-%m-%d %H:%M:%S}'.format(startTime)+"-07"
            startTime = '{:%Y-%m-%d %H:%M}'.format(startTime)+"-07"
    elif len(startTime) >= 6:
        startTime = '2017-' + startTime + '-07'   
    return startTime, pileName, carTypeName, userNickName

In [8]:
# ##------- Every once a while runs it update station and pile fixed info -------##
# #--------------------------------------------#
# #------- Gather all stations  -------# ****NEED TO BE FIXED 09.17.2017
# station_info = Get_py.get_allStations(hdr)
# print(len(station_info))
# for station in station_info:
#     stationId = str(station['id'])  
# #     print(stationId)
#     #------- Gather individual stations details -------#
#     station_detailedInfo = Get_py.get_oneStation_APP(hdr, stationId)
#     conn = Insert_py.insert_singStation_details(conn, station_detailedInfo)
#     #------- Gather individual pile code -------#
#     piles_info = Get_py.get_piles(stationId, proxies_pool, hdr)
#     conn = Insert_py.insert_pilecode(conn, stationId, piles_info)
# #------- Gather inidividual pile details -------#
# #------- Request stationid which has user activities from db -------#
try:
    cur = conn.cursor()
    cur.execute("select distinct stationid from tbl_app_useractivity;")
    stationWithAct_list = cur.fetchall()
    cur.close()
except psycopg2.Error as e:
    print(e)
print(len(stationWithAct_list))
# #------- Request pilecode from db for states -------#
staid_list = []
for staid in stationWithAct_list:
    staid_list.append(staid[0])
try:
    cur = conn.cursor()
    sta_list = ['2b3a35d4-cb94-4b8f-a47a-bbb054754066', '1d02640a-7f14-4fc3-baaf-60754a38fff7',
                'ac6cb946-5fff-491b-859e-55cc5f701657', 'dd4a3ddc-e1b0-4e2e-b7ca-faef627f4e66',
                'b134a159-16bd-4f5d-af1b-cd42599a3bab', '33a3c031-0a3a-417d-ba1b-53549bb08dc2', 
                'e4758edb-1183-4c51-887b-a52ca7abef3d', '13bdab71-110b-4448-a3c4-5c3113183546']
    #1. beijing hangkong jingmi jixie yanjiusuo chongdianzhan --'2b3a35d4-cb94-4b8f-a47a-bbb054754066',
    #2. beijing xijiao bingguan chongdianzhan -- '1d02640a-7f14-4fc3-baaf-60754a38fff7
    #3. beijing yangguanglicheng jiudian chongdianzhan --'ac6cb946-5fff-491b-859e-55cc5f701657',
    #4. beijing rongtao gongsi chongdianzhan -- '32d317a9-3d0c-4458-8357-89a2b3b1cf94'
    #5. beijing huairou qu yanshui lu -- 'dd4a3ddc-e1b0-4e2e-b7ca-faef627f4e66'
    
    #6. shenzhen hengjiang dasha chongdianzhan -- 'b134a159-16bd-4f5d-af1b-cd42599a3bab'
    #7. shenzhen yinxing gongyeyuan chongdianzhan -- '33a3c031-0a3a-417d-ba1b-53549bb08dc2'
    #8. shenzhe wanke jiulongshan chongdianzhan -- 'e4758edb-1183-4c51-887b-a52ca7abef3d'
    #9. shenzhen jingyuan dasha chongdianzhan -- '13bdab71-110b-4448-a3c4-5c3113183546'
    stmt = "select pilecode from tbl_APP_pilecode where staid in %s"
    cur.execute(stmt, (tuple(sta_list),))
    
#     stmt = "select pilecode from tbl_APP_pilecode where staid = 'dd4a3ddc-e1b0-4e2e-b7ca-faef627f4e66'"
#     cur.execute(stmt)
#     cur.execute("select pilecode from tbl_APP_pilecode")
    pillcode_list = cur.fetchall()
    cur.close()
except psycopg2.Error as e:
    print(e)
print(len(pillcode_list))

# # ------- Requests stationID from db for user charging activities -------#
try:
    cur = conn.cursor()
    cur.execute("select stationid from tbl_app_eachstations;")
    station_list = cur.fetchall()
    cur.close()
except psycopg2.Error as e:
    print(e)
print(len(station_list))

# reportTime = datetime.datetime.now() + datetime.timedelta(minutes=15)
# i = 0
# for pile in pilecode_list:
#     #If encounter error, try one more request with a different proxy
#     try:
#         pile_detailedInfo, proxies_pool = Get_py.get_onePile_APP(F, hdr, pile[0], proxies_pool, time_out=1000)
#     except ValueError as e:
#         try:
#             pile_detailedInfo, proxies_pool = Get_py.get_onePile_APP(F, hdr, pile[0], proxies_pool, time_out=1000)
#         except ValueError as e:
#             pile_detailedInfo = None
#         except (requests.exceptions.ChunkedEncodingError) as e:
#             pile_detailedInfo = None
#     except (requests.exceptions.ChunkedEncodingError) as e:
#         try:
#             pile_detailedInfo, proxies_pool = Get_py.get_onePile_APP(F, hdr, pile[0], proxies_pool, time_out=1000)
#         except ValueError as e:
#             pile_detailedInfo = None
#         except (requests.exceptions.ChunkedEncodingError) as e:
#             pile_detailedInfo = None
#     if pile_detailedInfo is not None:
#         conn = Insert_py.insert_singPile_details(conn, pile_detailedInfo)
#     else:
#         print("Running None Func.")
#         conn = Insert_py.insert_singPile_none(conn, pile_detailedInfo, pile[0])
#     i += 1
#     if datetime.datetime.now() >= reportTime:
#         reportTime = datetime.datetime.now() + datetime.timedelta(minutes=15) 
#         print("Successfully run {} piles".format(i))
# #211 Running None Func.
# #71585 piles.
# #------- Request Shenzhen charging pilecode from db for states -------#
try:
    cur = conn.cursor()
#     stmt1 = "select stationid from tbl_app_eachstations where tbl_app_eachstations.location && ST_MakeEnvelope(115.37916, 39.47148, 117.15623, 40.81726, 4326);"    
    stmt1 = "select stationid from tbl_app_eachstations where tbl_app_eachstations.location && ST_MakeEnvelope(113.85955, 22.50563, 114.17335, 22.69960, 4326) intersect select distinct stationid from tbl_app_useractivity;"
    cur.execute(stmt1)
    sta_list = cur.fetchall()
    sta_list_clean = []
    for sta in sta_list:
        sta_list_clean.append(sta[0])
    stmt2 = "select pilecode from tbl_APP_pilecode where staid in %s"
    cur.execute(stmt2, (tuple(sta_list_clean),))
    shenzhen_pillcode_list = cur.fetchall()
    cur.close()
except psycopg2.Error as e:
    print(e)
print(len(shenzhen_pillcode_list))

4859
86
8866
926


In [9]:
# # #------- Update user activity -------#
# i = 0
# fail = 0
# hour_uni = u'\u5c0f\u65f6\u524d'
# minute_uni = u'\u5206\u949f\u524d'
# # proxies = proxy()
# # proxy_idx = random.randint(0,len(proxies)-1) 
# # os.environ["https_proxy"]=proxies[proxy_idx]['https']
# # os.environ["http_proxy"]=proxies[proxy_idx]['http']
# activity_dict = {}
# for idx, stationID in enumerate(station_list):
#     stationID = stationID[0]
# #     if len(proxies) == 1: #proxy pool dried out
# #         proxies = proxy()
    
# #     proxy_idx = random.randint(0,len(proxies)-1)
# #     os.environ["https_proxy"]=proxies[proxy_idx]['https']
# #     os.environ["http_proxy"]=proxies[proxy_idx]['http']
    
#     try_num = 2
# #     while try_num:
# #         if try_num < 2:
# #             print("Retry")
# #         try:
# #             chargeList_detailedInfo = Get_py.get_chargelist_APP(stationID,ChargeHist_hdr)
# #         except:
# #             print("Changing proxy")
# #             proxies.pop(proxy_idx)
# #             if len(proxies) == 1:
# #                 proxies = proxy()
# #                 proxy_idx = random.randint(0,len(proxies)-1)
# #             else:
# #                 proxy_idx = random.randint(0,len(proxies)-1)
# #             os.environ["https_proxy"]=proxies[proxy_idx]['https']
# #             os.environ["http_proxy"]=proxies[proxy_idx]['http']
# #         try_num -= 1
# #         if not try_num:
# #             chargeList_detailedInfo=[]
# #             fail += 1
# #             print('fail +', fail, ' index', idx)
                
#     try:
#         chargeList_detailedInfo = Get_py.get_chargelist_APP(stationID,ChargeHist_hdr)
#     except:
#         try:
# #             proxies.pop(proxy_idx)
# #             proxy_idx = random.randint(0,len(proxies)-1)
# #             os.environ["https_proxy"]=proxies[proxy_idx]['https']
# #             os.environ["http_proxy"]=proxies[proxy_idx]['http']
#             chargeList_detailedInfo = Get_py.get_chargelist_APP(stationID,ChargeHist_hdr)
#         except:
# #             proxies.pop(proxy_idx)
#             chargeList_detailedInfo=[]
#             fail += 1
#             print('fail +', fail, ' index', idx)
#             pass
#     if len(chargeList_detailedInfo) == 0:
#         i += 1
#     else:
#         activity_dict['{}'.format(stationID)] = chargeList_detailedInfo
# print(str(i) + " stations do not have activity history.")
# print(str(fail) + " connections failed attempt.")
# print(datetime.datetime.now())
# fromaddr = "296257740.jz@gmail.com"
# psw = "15710725968crzzq"
# toaddr = "hustlejanton@berkeley.edu"
# subject="Courtesy Email"
# body="Program terminated by time - 16046 user activity"
# reminder.email_courtesy(fromaddr, toaddr, psw, subject, body)
# print("Email sent successfully.")

In [10]:
# for stationID in activity_dict.keys():
#     chargeList_detailedInfo = activity_dict[stationID]
#     for user_act in chargeList_detailedInfo:
#         startTime, pileName, carTypeName, userNickName = user_act_AIclean(user_act)
#         conn = Insert_py.insert_userActivity_info(conn, startTime, pileName, stationID, carTypeName, userNickName)

In [9]:
def pillState_scrap(pillcode): 
    """
        Input:
            pillCode - string, each charging pile has its own pile code
        Output:
            pill_detailedInfo - dictionary of pill state details
    """
    hdr = header_gen()
    time_out = 20
    F = None
    proxies_pool = None
    #If encounter error, try one more request with a different proxy
    try:
        pill_detailedInfo, proxies_pool = Get_py.get_onePill_APP(hdr, pillcode, F, proxies_pool, time_out=time_out)
#     except ValueError as e:
#         if F is not None:
#             write_error(F, pillcode, e)
#         try:
#             pill_detailedInfo, proxies_pool = Get_py.get_onePill_APP(hdr, pillcode, F, proxies_pool, time_out=time_out)
#         except ValueError as e:
#             if F is not None:
#                 write_error(F, pillcode, e)
#             pill_detailedInfo = None
#         except (requests.exceptions.ChunkedEncodingError) as e:
#             if F is not None:
#                 write_error(F, pillcode, e)
# #             else:
#             pill_detailedInfo = None
#     except (requests.exceptions.ChunkedEncodingError) as e:
#         if F is not None:
#             write_error(F, pillcode, e)
#         try:
#             pill_detailedInfo, proxies_pool = Get_py.get_onePill_APP(hdr, pillcode, F, proxies_pool, time_out=time_out)
#         except ValueError as e:
#             if F is not None:
#                 write_error(F, pillcode, e)
#             pill_detailedInfo = None
#         except (requests.exceptions.ChunkedEncodingError) as e:
#             if F is not None:
#                 write_error(F, pillcode, e)
# #         else:
#             pill_detailedInfo = None
# #     return pill_detailedInfo, proxies_pool
    except:
        pill_detailedInfo = None
    return pill_detailedInfo

In [10]:
def insert_state(outputs):
    for pill_detailedInfo in outputs:
        if pill_detailedInfo is not None:
            db_name="db_teld"
            user_name="postgres"
            password = "ecalscraping"
            host="127.0.0.1"
            port="5432"
            conn = indb.init_db(db_name, user_name, password, host, port)
            pillCode = str(pill_detailedInfo['pillCode'])
            stateCode = str(pill_detailedInfo['stateCode'])
            stateCodeV3 = str(pill_detailedInfo['stateCodeV3'])
            if pill_detailedInfo['stateName'] is not None:
                stateName = str(pill_detailedInfo['stateName'].encode('utf-8'))
            else:
                stateName = ''
            if pill_detailedInfo['stateNameV3'] is not None:
                stateNameV3 = str(pill_detailedInfo['stateNameV3'].encode('utf-8'))
            else:
                stateNameV3 = ''
            conn = Insert_py.insert_state_info(conn, pillCode, stateCode, stateName, stateCodeV3, stateNameV3)
            conn.close()

In [13]:
## ----- Test for single looping of multiprocessing post_request and insert -----##

# pid = os.getpid()
# py = psutil.Process(pid)
# memoryUse = py.memory_info()[0]/2.**20 #in MB
# print('Before runnning script, memory use:', memoryUse, 'MB')
# # loop_num = 1
# # max_proc =10000
# pool = Pool(200)
# t = time.time()
# if __name__ == '__main__':
#     results = [pool.apply_async(pillState_scrap, (pillcode[0],)) for pillcode in pillcode_list]
#     print(len(results))
#     outputs = [p.get() for p in results]
# print("success, time costs is " + str(time.time()-t)) 

# pid = os.getpid()
# py = psutil.Process(pid)
# memoryUse = py.memory_info()[0]/2.**20 #in MB
# print('After runnning script,memory use:', memoryUse, 'MB')

In [11]:
for pillcode in pillcode_list[70:]:
    print(pillcode[0])

4403060016101
4403060072102
4403060072110
4403060072112
4403060072104
4403060072106
4403060072107
4403060072113
4403060072111
4403060072103
4403060072108
4403060072109
4403060072101
4403060072115
4403060072105
4403060072114


In [15]:
# os.environ["https_proxy"]="https://165.84.167.54:8080"
# os.environ["http_proxy"]="http://165.84.167.54:8080"

In [ ]:
# pool = Pool(processes=2)
# startTime = datetime.datetime.now()
endTime = datetime.datetime.now() + datetime.timedelta(hours=168)
# proxyUpdateTime = datetime.datetime.now() + datetime.timedelta(hours=3)
# tmpproxy = {'http': 'http://101.78.219.178:8080', 'https': 'https://101.78.219.178:8080'}
# os.environ["https_proxy"]=tmpproxy['https']
# os.environ["http_proxy"]=tmpproxy['http']
# proxies = proxy()
# proxy_idx = random.randint(0,len(proxies)-1) 
# os.environ["https_proxy"]=proxies[proxy_idx]['https']
# os.environ["http_proxy"]=proxies[proxy_idx]['http']

fromaddr = "296257740.jz@gmail.com"
psw = "15710725968crzzq"
toaddr = "hustlejanton@berkeley.edu"
subject="Courtesy Email"
while True:
    if datetime.datetime.now() >= endTime:
        body="Program terminated by time - 16043"
        reminder.email_courtesy(fromaddr, toaddr, psw, subject, body)
        break
#     Tor.renewTor()
#     Tor.connectTor()
#     Tor.showmyIp()
    print("waking up!")
#     if datetime.datetime.now() >= proxyUpdateTime: #hit update time
#         proxies = proxy()
#         proxyUpdateTime = datetime.datetime.now() + datetime.timedelta(hours=3)
    if len(proxies) <= 1: #proxy pool dried out before next update time
        proxies = proxy()
        proxy_idx = random.randint(0,len(proxies)-1) 
        os.environ["https_proxy"]=proxies[proxy_idx]['https']
        os.environ["http_proxy"]=proxies[proxy_idx]['http']
    t = time.time()
    outputs=[]
#     results = [pool.apply_async(pillState_scrap, (pillcode[0],)) for pillcode in pillcode_list]
#     outputs = [p.get() for p in results]
    missing_data = 0
    for pillcode in pillcode_list:
#         Tor.connectTor()
        try_num = 2
        while try_num:
            if try_num < 2:
                print("Retry")
            pill_detailedInfo = pillState_scrap(pillcode[0])
            if pill_detailedInfo is not None:
                outputs.append(pill_detailedInfo)
                break
            if pill_detailedInfo is None:
                print("Changing proxy")
                proxies.pop(proxy_idx)
                if len(proxies) == 1:
                    proxies = proxy()
                    proxy_idx = random.randint(0,len(proxies)-1)
                else:
                    proxy_idx = random.randint(0,len(proxies)-1)
                os.environ["https_proxy"]=proxies[proxy_idx]['https']
                os.environ["http_proxy"]=proxies[proxy_idx]['http']
            try_num -= 1
            if not try_num:
                missing_data += 1
    if endTime - datetime.datetime.now() <= datetime.timedelta(hours=1):
        body="Program less than one hour left."
        reminder.email_courtesy(fromaddr, toaddr, psw, subject, body)
    print("Takes "+str(time.time()-t)+" seconds")
    print("Missing {} of pills data".format(missing_data))
    insert_state(outputs)
    print("sleeping...")
    time.sleep(random.randint(600, 900))

waking up!
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db624e50>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbaf2ad0>: Failed to establish a new c

HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7adb50>, 'Connection to 13.125.169.49 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 500 Internal Privoxy Error',)))
failed
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Timeout occured
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db32ca90>, 'Connection to 36.67.45.95 timed out. (connect timeout

HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbb787d0>, 'Connection to 5.189.129.146 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError

HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db5a0150>, 'Connection to 122.183.137.190 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7e57f4850>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max

HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db850a90>, 'Connection to 207.148.4.234 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db5a07d0>, 'Connection to 139.224.80.139 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db5a0350>, 'Connection to 121.199.42.198 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy

...

Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7ac450>, 'Connection to 219.155.131.207 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7acd10>, 'Connection to 51.15.90.184 timed out. (connect timeout=20)'))
failed
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7a29d0

HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9b1a10>, 'Connection to 118.69.61.212 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9b15d0>, 'Connection to 89.186.1.215 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9b1690>, 'Connection to 119.42.75.26 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTT

Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7dbaf9310>, 'Connection to 94.253.106.3 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db956510>, 'Connection to 191.248.189.181 timed out. (connect timeout=20)'))
failed
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
Takes 523.626106024 seconds
Missing 7 of pills data
sleeping...
waking up!
Changing proxy
Retry
Changing proxy
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=44

Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db9cf290>, 'Connection to 179.125.127.90 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Too many open connections',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db956510>, 'Co

HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 503 Too many open connections',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa7db7adc10>, 'Connection to 187.188.137.148 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by 

In [ ]:
# ## pid = os.getpid()
# py = psutil.Process(pid)
# memoryUse = py.memory_info()[0]/2.**20 #in MB
# print('Before runnning script, memory use:', memoryUse, 'MB')
# pool = Pool(100)
# t = time.time()
# if __name__ == '__main__':
#     results = [pool.apply_async(insert_state, (pill_detailedInfo,)) for pill_detailedInfo in outputs]
#     print(len(results))
#     outputs = [p.get() for p in results]
# print("success, time costs is " + str(time.time()-t)) 
# pid = os.getpid()
# py = psutil.Process(pid)
# memoryUse = py.memory_info()[0]/2.**20 #in MB
# print('After runnning script, memory use:', memoryUse, 'MB')## pid = os.getpid()
# py = psutil.Process(pid)
# memoryUse = py.memory_info()[0]/2.**20 #in MB
# print('Before runnning script, memory use:', memoryUse, 'MB')
# pool = Pool(100)
# t = time.time()
# if __name__ == '__main__':
#     results = [pool.apply_async(insert_state, (pill_detailedInfo,)) for pill_detailedInfo in outputs]
#     print(len(results))
#     outputs = [p.get() for p in results]
# print("success, time costs is " + str(time.time()-t)) 
# pid = os.getpid()
# py = psutil.Process(pid)
# memoryUse = py.memory_info()[0]/2.**20 #in MB
# print('After runnning script, memory use:', memoryUse, 'MB')